# Connect and Authenticate with a Data Owner

As a Data Scientist, you want to be able to connect with data owners and verify that they actually have the data that you are looking for before engaging in any privacy-preserving machine learning flows.

In this notebook you will learn how to do that, requesting the data owner proves that they have a Data Owner credential issued from the OM Authority.

**You Should have started with the Data Owner. Port 8889**

## 5. Initialise the Data Scientist Controller

We will just use the basic controller on it's own in this notebook so this code should feel quite familiar. We will not be managing a list of trusted connections in this notebook, instead we will be focusing on the present-proof protocol in detail.

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8062
WEBHOOK_BASE = ""
ADMIN_URL = "http://datascientist-agent:8061"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 6. Listen to Webhooks and Register Listeners

In these listeners we dig into the messages getting communicated between agents as part of the present-proof protocol. Similar to the issue-credential protocol in part 6.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())


def proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    

    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Object\n")
        print(presentation_request)
        print("\nName : ", presentation_request["name"])
        print("\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n")
        print(presentation_request["requested_attributes"])
        print("\nThere is also a requested_predicates object, which is empty because we did not request any")
        print("\nAnd as with issue credential we have a nonce, which gives the issuer confidence in the liveness of the presentation\n")
        print(presentation_request["nonce"])
    elif state == "presentation_sent":
        
        print("The Presentation object is a bit overwhelming. Let's look at it in detail\n")
        presentation = payload["presentation"]
        
        print("\nThe Requested Proof object contains revealed, unrevealed, self-attested and predicate attributes\n")
        print(presentation["requested_proof"])
        print("\nThe revealed attributes contains the raw and encoded version of the scope attribute. As well as an index into the proof object we will look at later.\n")
        print(presentation["requested_proof"]["revealed_attrs"])
        
        print("\nAll identifiers associated with the presentation")
        print(presentation["identifiers"])
        
        print("\nThen the Proof object - the crypto bit\n")
        proof = presentation["proof"]
        print(proof)
        
        print("\nThis contains an array of proofs, one element for each attribute disclosed (I think)\n")
        print(proof["proofs"])
        
        print("\nEach proof element contains a primary proof and a proof of non-revocation (which will be empty)\n")
        proof_elem = proof["proofs"][0]
        print(proof_elem)
        
        print("\nEach primary proof contains an eq_proof, which is where all the numbers are for the crypto proving an attribute disclosed really is equal to the one signed\n")
        print(proof_elem["primary_proof"]["eq_proof"])
        
        print("\nFinally the proofs are combined into a single aggregated proof object\n")
        print(proof["aggregated_proof"])
        
        print("\nNotice again the c_hash, this is the challenge for the non-interactive ZKP proof which is verifying this presentation\n")
        
    
        
        
        
    elif state == "presentation_acked":
        print("Presentation has been acknowledged by the Issuer")
    elif state == "request_sent":
        print("Presentation Request\n")
        print(payload["presentation_request"])
        print("\nThe presentation request is encoded in base64 and packaged into a DIDComm Message\n")
        print(payload["presentation_request_dict"])
        print("\nNote the type defines the protocol present-proof and the message request-presentation\n")
        print(payload["presentation_request_dict"]["@type"])
    elif state == "presentation_received":
        print("Presentation Received")
        print("We will not go into detail on this payload as it is comparable to the presentation_sent we looked at in the earlier cell.")
        print("This is the full payload\n")
        print(payload)
    else:
        print("Paload \n")
        print(payload)
    

proof_listener = {
    "handler": proof_handler,
    "topic": "present_proof"
}   

def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    state = payload['state']
    print("State", state)
    if state == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([proof_listener,connection_listener], defaults=True)

## 7 Accept Invitation from Data Owner

In [3]:
invite = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "bd5a48d8-11be-4e17-a880-74e4cea6a45a", "serviceEndpoint": "https://e695090e077b.ngrok.io", "recipientKeys": ["Agp992DQggeQLb17a7LPCn4w64WevuTDr7mkPtnAvn2c"], "label": "DATAOWNER"}

### BE SURE TO SCROLL THROUGH THE OUTPUT TO SEE A BREAKDOWN OF THE AGENT MESSAGES EXCHANGED

In [4]:
response = await agent_controller.connections.accept_connection(invite)

connection_id = response["connection_id"]

Handle Connection Webhook Payload
Connection ID 187f3026-3e5d-4684-9b92-3fb817fc93c9
State invitation
Handle Connection Webhook Payload
Connection ID 187f3026-3e5d-4684-9b92-3fb817fc93c9
State request
Handle Connection Webhook Payload
Connection ID 187f3026-3e5d-4684-9b92-3fb817fc93c9
State response
Handle Connection Webhook Payload
Connection ID 187f3026-3e5d-4684-9b92-3fb817fc93c9
State active
Connection 187f3026-3e5d-4684-9b92-3fb817fc93c9 changed state to active

---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  d9d227c7-e9ca-4ffd-8898-ca42064717ab
Protocol State :  request_received
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

Recieved Presentation Object

{'name': 'Proof of Scientist', 'version': '1.0', 'requested_attributes': {'0_scope_uuid': {'name': 'scope', 'restrictions': [{'schema_id'


---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  d9d227c7-e9ca-4ffd-8898-ca42064717ab
Protocol State :  presentation_acked
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

Presentation has been acknowledged by the Issuer


## 8. Review the Output in the Data Owner

Then come back here to challenge the data owner to prove that they are indeed a dataowner.

## 9. Send Challenge to Data Owner

As a Data Scientist you want to have confidence that the Data Owner is who they claim to be, that they have the dataset you are looking for and potentially any other number of verifiable information flows.

For this tutorial we will just request proof that they have a Data Owner credential from the OM Authority.

### 9.1. Define Trusted Issuer

You first need to copy the OM Authority DID you registered on the ledger in Notebook 3. Step 8. and use it in the proof request object below.

In [5]:
#####################################################
## YOU NEED TO CHANGE THIS TO YOUR OM AUTHORITY DID
trusted_issuer_did = "vrzjfm1MEN1g5o6QtHLfv"
#####################################################

### 9.2. Define Presentation Request Attributes and Constraints

In [6]:
# We add a constraint that the attribute must originate from this schema
schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1"



# You could additionally specify the cred_def id you wish. 
# You would have to copy this from the OM Authority notebook.
# cred_def = "WfntKNFwXMQfgmU9ofbxPM:3:CL:156569:default"

revocation = False
exchange_tracing = False

# We are the Data Owner for their name and the domain of the data that they have
req_attrs = [
    {"name": "domain", "restrictions": [{"schema_id": schema_id, "issuer_did": trusted_issuer_did}]},#, "cred_def_id": cred_def}]},
    {"name": "name", "restrictions": [{"schema_id": schema_id, "issuer_did": trusted_issuer_did}]}
]

# We could extend this to request the name attribute aswell if we wanted.


indy_proof_request = {
    "name": "Proof of Data Owner",
    "version": "1.0",
    "requested_attributes": {
        # They must follow this uuid pattern
        # Note that req_attr['name'] gets the attribute name of each object. E.g. domain and name in this case
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}

print(indy_proof_request)


{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_domain_uuid': {'name': 'domain', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}, '0_name_uuid': {'name': 'name', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}}, 'requested_predicates': {}}


### 9.3. Specify Connection ID and Send Request

**AGAIN, MAKE SURE YOU SCROLL THROUGH THE OUTPUT TO SEE THE PRESENTATION PROTOCOL FROM THE VERIFIER SIDE**

In [7]:
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": False
}

# Send proof request
response = await agent_controller.proofs.send_request(proof_request_web_request)

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/aries_basic_controller/aries_controller.py", line 123, in _receive_webhook
    await self._handle_webhook(topic, payload)
  File "/aries_basic_controller/aries_controller.py", line 128, in _handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendMessage
    listener(data, self, allData)
  File "/o


---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  938b2d9d-a433-47e8-9feb-294b355d5a27
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_domain_uuid': {'name': 'domain', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}, '0_name_uuid': {'name': 'name', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}}, 'requested_predicates': {}, 'nonce': '846385465499850757613816'}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/aries_basic_controller/aries_controller.py", line 123, in _receive_webhook
    await self._handle_webhook(topic, payload)
  File "/aries_basic_controller/aries_controller.py", line 128, in _handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendMessage
    listener(data, self, allData)
  File "/o


---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  938b2d9d-a433-47e8-9feb-294b355d5a27
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_domain_uuid': {'name': 'domain', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}, '0_name_uuid': {'name': 'name', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}}, 'requested_predicates': {}, 'nonce': '846385465499850757613816'}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/aries_basic_controller/aries_controller.py", line 123, in _receive_webhook
    await self._handle_webhook(topic, payload)
  File "/aries_basic_controller/aries_controller.py", line 128, in _handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendMessage
    listener(data, self, allData)
  File "/o


---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  938b2d9d-a433-47e8-9feb-294b355d5a27
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_domain_uuid': {'name': 'domain', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}, '0_name_uuid': {'name': 'name', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}}, 'requested_predicates': {}, 'nonce': '846385465499850757613816'}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/aries_basic_controller/aries_controller.py", line 123, in _receive_webhook
    await self._handle_webhook(topic, payload)
  File "/aries_basic_controller/aries_controller.py", line 128, in _handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendMessage
    listener(data, self, allData)
  File "/o


---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  938b2d9d-a433-47e8-9feb-294b355d5a27
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_domain_uuid': {'name': 'domain', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}, '0_name_uuid': {'name': 'name', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}}, 'requested_predicates': {}, 'nonce': '846385465499850757613816'}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/aries_basic_controller/aries_controller.py", line 123, in _receive_webhook
    await self._handle_webhook(topic, payload)
  File "/aries_basic_controller/aries_controller.py", line 128, in _handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendMessage
    listener(data, self, allData)
  File "/o


---------------------------------------------------------------------

Handle present-proof
Connection ID :  187f3026-3e5d-4684-9b92-3fb817fc93c9
Presentation Exchange ID :  938b2d9d-a433-47e8-9feb-294b355d5a27
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_domain_uuid': {'name': 'domain', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}, '0_name_uuid': {'name': 'name', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1', 'issuer_did': 'vrzjfm1MEN1g5o6QtHLfv'}]}}, 'requested_predicates': {}, 'nonce': '846385465499850757613816'}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',

## 10. Verify the Presentation

Once a presentation has been received you still need to request your agent verify the cryptography to ensure the integrity of the information presented.

In [ ]:
presentation_exchange_id = response["presentation_exchange_id"]

verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

## 11. Parse Verified Presentation Object

Let's look at how we check the result of the verification and how you can access the revealed attributes. Maybe you need them for your business use case, perhaps they specify the type or quality of data the Data Owner might have. Who knows, it is up to you to define the information flows to meet your needs.

### 11.1. WARNING. The state is note an indicator of successful verification.

The presentation exchange object is in the verified state. That just means you have requested your agent to verify the cryptography, not that the cryptography is verified.

In [ ]:
print(verify['state'])

### 11.2. Check the Verified property

It should be true if the cryptographic presentation is verifiable

In [ ]:
print(verify['verified'])

### 11.3 Lets look at the revealed attributes

In [ ]:
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("Attribute : ", val)
    print("Raw Value : ", val['raw'])

## End of Notebook

In [ ]:
await agent_controller.terminate()

## Congratulations Tutorial Series Complete

Well done! Make sure you review the output in the Data Owner notebook and terminate the controller.

The final aspect of this course will involve the application of what we have learnt in this notebook series to mediate a PySyft Duet Learning flow.